# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-06-07&until-occurred-date=2021-06-14


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

15598 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-07&until-occurred-date=2021-06-14
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=f4e6d46c-29d4-4cc7-a59c-e640958e5238&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-07&until-occurred-date=2021-06-14
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=5f4f68d8-c54e-4c61-bd9c-355a8f07eeea&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-07&until-occurred-date=2021-06-14
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=ea6c1e0e-dfdf-4e82-8e34-d6350ad0cad4&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-05-10.csv
failed to load preprint_tweets_2021-05-17.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load tweets_sorted.json
failed to load preprint_tweets_2021-06-07.csv
failed to load preprint_tweets_2021-05-18.csv
failed to load preprint_tweets_2021-05-24.csv
failed to load preprint_tweets_2021-05-31.csv
failed to load preprint_tweets_2021-05-09.csv
failed to load preprint_tweets_2021-05-08.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 5]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []

for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.01.27.21250604...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21257987...
REST API query complete  200
REST API query started for 10.1101/2021.05.30.21257971...
REST API query complete  200
REST API query started for 10.1101/2021.02.23.432474...
REST API query complete  200
REST API query started for 10.1101/2020.11.15.383323...
REST API query complete  200
REST API query started for 10.1101/2020.12.02.403477...
REST API query complete  200
REST API query started for 10.1101/2021.05.22.21257658...
REST API query complete  200
REST API query started for 10.1101/2021.05.19.21257334...
REST API query complete  200
REST API query started for 10.1101/2021.03.05.21252960...
REST API query complete  200
REST API query started for 10.1101/2021.03.01.212

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.06.09.447807,40,bioRxiv,Neuroscience,False,Imaging of the pial arterial vasculature of th...,"[{'name': 'Saskia Bollmann'}, {'name': 'Hendri...",<p>The pial arterial vasculature of the human ...,2021-06-10
1,10.1101/2021.06.12.448192,40,bioRxiv,Microbiology,False,Harnessing methylotrophs as a bacterial platfo...,"[{'name': 'Nathan Michael Good'}, {'name': 'Ha...",<p>Gadolinium is a key component of magnetic r...,2021-06-12
2,10.1101/2021.06.06.447246,37,bioRxiv,Genetics,False,A single amino acid in RTEL1 determines telome...,"[{'name': 'Riham Smoom'}, {'name': 'Catherine ...",<p>Maintaining optimal telomere length through...,2021-06-07
3,10.1101/2021.06.09.447813,35,bioRxiv,Neuroscience,False,Toward a More Accurate 3D Atlas of C. elegans ...,"[{'name': 'Michael Skuhersky'}, {'name': 'Tail...",<p>Determining cell identity in volumetric ima...,2021-06-10
4,10.1101/2021.06.11.448148,30,bioRxiv,Neuroscience,False,Developmental Regulation of Homeostatic Plasti...,"[{'name': 'Wei Wen'}, {'name': 'Gina Turrigian...",<p>Homeostatic plasticity maintains network st...,2021-06-12
5,10.1101/2021.06.08.447573,29,bioRxiv,Plant Biology,False,Endosperm evolution by duplicated and neofunct...,"[{'name': 'Yichun Qiu'}, {'name': 'Claudia Köh...",<p>MADS-box transcription factors (TFs) are pr...,2021-06-09
6,10.1101/2021.06.07.447437,26,bioRxiv,Microbiology,True,FXa cleaves the SARS-CoV-2 spike protein and b...,"[{'name': 'Wenjuan Dong'}, {'name': 'Jing Wang...",<p>The ongoing coronavirus disease 2019 (COVID...,2021-06-08
7,10.1101/2021.06.08.447593,23,bioRxiv,Cell Biology,False,ER-lysosome lipid transfer protein VPS13C/PARK...,"[{'name': 'William Hancock-Cerutti'}, {'name':...","<p>Mutations in VPS13C cause early onset, auto...",2021-06-08
8,10.1101/2021.06.07.447379,22,bioRxiv,Immunology,False,Hypoxia is a dominant remodeler of the CD8+ T ...,"[{'name': 'James R. Byrnes'}, {'name': 'Amy M....",<p>Immunosuppressive factors in the tumor micr...,2021-06-07
9,10.1101/2021.06.09.447744,21,bioRxiv,Synthetic Biology,False,A light tunable differentiation system for the...,"[{'name': 'Chetan Aditya'}, {'name': 'François...",<p>Artificial microbial consortia seek to leve...,2021-06-09


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 5 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

43 preprints (including 9 covering SARS-CoV-2, 39 from bioRxiv and 4 from medRxiv) published in the last 7 days before 2021-06-14 had been tweeted at least 5 times (maximum 40).